In [37]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import random

In [38]:
url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

In [39]:
r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': 2})
print(r)

<Response [200]>


In [40]:
soup = BeautifulSoup(r.text, 'html.parser')
string = 'Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies'
string in soup.find('body').text

False

In [5]:
product_links = pd.DataFrame(columns=['product', 'link'])
i = 0

url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

for page in range(1, 1000):

    print("scraping page - ", page)
    
    r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': 3})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # stop if next page button is disabled
    if soup.find('li', {'class': 'a-disabled a-last'}):
        break
    else:
        # go to next page
        # find element of next page
        next_links = soup.find_all('li', {'class':'a-last'})
        for link in next_links:
            url = 'https://amazon.com' + str(link.find('a')['href'])
    
    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in products:
        item = product.find('a', {'class':'a-link-normal s-underline-text s-underline-link-text a-text-normal'})
        if item == None:
            item = product.find('a', {'class':'a-link-normal a-text-normal'})
        title = item.text.strip()
        link = item['href']
        product_links.loc[i] = [title, link]
        i = i+1
    if not page == 1:
        soup = soup.find_all('li', {'class':'a-normal'})

print('done')
product_links.to_csv('scraped_product_links.csv', index=False)

scraping page -  1
scraping page -  2
scraping page -  3
scraping page -  4
scraping page -  5
scraping page -  6
scraping page -  7
done


In [9]:
products = pd.read_csv('products_complete.csv')

#products['features'] = ''
#products['description'] = ''
#products['review_pages'] = ''

for i in range(110, len(products)):
    
    print("", round((100 * i) / len(products)), "% product pages scraped")
    
    url_base = products['link'].iloc[i]
    url = 'https://amazon.com' + url_base

    r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': random.randint(2,20)})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        if str(r) == "<Response [503]>" or string in soup.find('body').text:
            print('#####amazon is blocking you######')
            print('no of pages scraped - ',i )
            break
    except:
        print('#####amazon is blocking you######')
        print('no of pages scraped - ',i )
        break

    product_features = ''
    product_description = ''
    review_page_link = ''
    
    review_item = soup.find({'a'}, {'data-hook':"see-all-reviews-link-foot"})
    
    if review_item is None:          
        products['features'][i] = product_features
        products['description'][i] = product_description
        products['review_pages'][i] = review_page_link
        print('Not enough reviews - adding empty entries')
        continue
        
    review_page_link = review_item['href']

    feature_list_conatiner = soup.find({'div'}, {'id':"feature-bullets"})
    feature_list = feature_list_conatiner.find_all({'span'}, {'class':"a-list-item"})
    for feature in feature_list:
        product_features = product_features + " " + feature.text.strip()

    product_description_container = soup.find('div', 
                                              {'cel_widget_id':
                                               "dpx-aplus-3p-product-description_csm_instrumentation_wrapper"})    
    
    if product_description_container == None:
        product_description_container = soup.find('div', {'id':'productDescription'})

    if product_description_container == None:
        products['features'][i] = product_features
        products['description'][i] = product_description
        products['review_pages'][i] = review_page_link
        continue
    
    lists = product_description_container.find_all({'span'}, {'class':"a-list-item"})
    
    for item in lists:
        product_description = product_description + " " + item.text.strip()

    paras = product_description_container.find_all('p')
    for para in paras:
        if '<img alt=' in para.text:
            continue
        product_description = product_description + " " + para.text.strip()
    
    products['features'][i] = product_features
    products['description'][i] = product_description
    products['review_pages'][i] = review_page_link

 30 % product pages scraped
 30 % product pages scraped
 31 % product pages scraped
#####amazon is blocking you######
no of pages scraped -  110


In [10]:
soup

{"error": 504, "type": "GlobalTimeoutError", "description": "Timeout exceeded rendering page", "info": {"remaining": -0.013931035995483398, "timeout": 30}}

In [11]:
i

110

In [12]:
products.to_csv('products_complete.csv', index = False)

In [42]:
def clean_title(item):
    if item == None:
        return ""
    else:
        return item.text.strip().replace('\n', '')
    
def clean_rating(item):
    if item == None:
        return 0.0
    else:
        return float(item.text[:3])

def clean_body(item):
    if item == None:
        print("empty body")
        return ""
    else:
        return item.text.strip()
#df = pd.DataFrame(columns = ['product', 'review_title', 'review_rating', 'review_text'])
df = pd.read_csv('scraped_data1.csv')


#first 450 should be scraped again
products = pd.read_csv('products_complete.csv')
i = len(df)

for prd_idx in range(39, len(products)):
    print('scraping reviews of product - ', prd_idx + 1)
    url_base = products['review_pages'][prd_idx]
    
    if url_base == "":
        continue
    if type(url_base) == type(2.0):
        continue
    
    for page in range(1,15):
        print('reviews scrapped - ', i)
        url = 'https://amazon.com' + url_base + str(page)
        r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': random.randint(2,20)})
        
        soup = BeautifulSoup(r.text, 'html.parser')
        
        try:
            if str(r) == "<Response [503]>" or string in soup.find('body').text:
                print('#####amazon is blocking you######')
                print('no of pages scraped - ',i )
                break
        except:
            print('#####amazon is blocking you######')
            print('no of pages scraped - ',i )
            break

        #check if we have reached last page
        if soup.find('li', {'class': 'a-disabled a-last'}):
            break

        reviews = soup.find_all('div', {'data-hook': 'review'})

        for items in reviews:
            product = soup.title.text.replace('Amazon.com: Customer reviews:', '').strip()
            reviews_title= clean_title(items.find('a', {'data-hook': 'review-title'}))
            review_rating= clean_rating(items.find('i', {'data-hook': 'review-star-rating'}))
            reviews_text= clean_body(items.find('span', {'data-hook': 'review-body'}))
            df.loc[i] = [product, reviews_title, review_rating, reviews_text]
            i = i + 1

print("Done scraping")        
df.to_csv('scraped_data.csv')

scraping reviews of product -  40
scraping  4640  review page of product -  40
scraping  4650  review page of product -  40
scraping  4660  review page of product -  40
scraping  4670  review page of product -  40
scraping  4680  review page of product -  40
scraping  4690  review page of product -  40
scraping  4700  review page of product -  40
scraping  4710  review page of product -  40
scraping  4720  review page of product -  40
scraping  4730  review page of product -  40
scraping  4740  review page of product -  40
scraping  4750  review page of product -  40
scraping  4760  review page of product -  40
scraping  4770  review page of product -  40
scraping reviews of product -  41
scraping  4780  review page of product -  41
scraping  4790  review page of product -  41
scraping  4800  review page of product -  41
scraping  4810  review page of product -  41
scraping  4820  review page of product -  41
scraping  4830  review page of product -  41
scraping  4840  review page of p

scraping  6280  review page of product -  54
scraping  6290  review page of product -  54
scraping reviews of product -  55
scraping  6300  review page of product -  55
scraping  6310  review page of product -  55
scraping  6320  review page of product -  55
scraping  6330  review page of product -  55
scraping  6340  review page of product -  55
scraping  6350  review page of product -  55
scraping  6360  review page of product -  55
scraping  6370  review page of product -  55
scraping  6380  review page of product -  55
scraping  6390  review page of product -  55
scraping  6400  review page of product -  55
scraping  6410  review page of product -  55
scraping  6420  review page of product -  55
scraping  6430  review page of product -  55
scraping reviews of product -  56
scraping  6440  review page of product -  56
scraping  6450  review page of product -  56
scraping  6460  review page of product -  56
scraping  6470  review page of product -  56
scraping  6480  review page of p

scraping  7836  review page of product -  69
scraping  7846  review page of product -  69
scraping  7856  review page of product -  69
scraping  7866  review page of product -  69
scraping  7876  review page of product -  69
scraping  7886  review page of product -  69
scraping  7896  review page of product -  69
scraping  7906  review page of product -  69
scraping  7916  review page of product -  69
scraping  7926  review page of product -  69
scraping  7936  review page of product -  69
scraping  7946  review page of product -  69
scraping  7956  review page of product -  69
scraping reviews of product -  70
scraping  7966  review page of product -  70
scraping  7976  review page of product -  70
scraping  7986  review page of product -  70
scraping  7996  review page of product -  70
scraping  8006  review page of product -  70
scraping  8016  review page of product -  70
scraping  8026  review page of product -  70
scraping  8036  review page of product -  70
scraping  8046  revie

scraping  9220  review page of product -  83
scraping  9230  review page of product -  83
scraping  9240  review page of product -  83
scraping  9250  review page of product -  83
scraping  9260  review page of product -  83
scraping  9270  review page of product -  83
scraping  9280  review page of product -  83
scraping  9290  review page of product -  83
scraping  9300  review page of product -  83
scraping reviews of product -  84
scraping  9310  review page of product -  84
scraping  9320  review page of product -  84
scraping  9330  review page of product -  84
scraping  9340  review page of product -  84
scraping  9350  review page of product -  84
scraping  9360  review page of product -  84
scraping  9370  review page of product -  84
scraping  9380  review page of product -  84
scraping  9390  review page of product -  84
scraping  9400  review page of product -  84
scraping  9410  review page of product -  84
scraping  9420  review page of product -  84
scraping  9430  revie

scraping  10930  review page of product -  96
scraping  10940  review page of product -  96
scraping  10950  review page of product -  96
scraping  10960  review page of product -  96
scraping  10960  review page of product -  96
scraping  10970  review page of product -  96
scraping reviews of product -  97
scraping  10980  review page of product -  97
scraping  10990  review page of product -  97
scraping  11000  review page of product -  97
scraping  11010  review page of product -  97
scraping  11020  review page of product -  97
scraping  11030  review page of product -  97
scraping  11040  review page of product -  97
scraping  11050  review page of product -  97
scraping  11060  review page of product -  97
scraping  11070  review page of product -  97
scraping  11080  review page of product -  97
scraping  11090  review page of product -  97
scraping  11100  review page of product -  97
scraping  11110  review page of product -  97
scraping reviews of product -  98
scraping  11

scraping  12478  review page of product -  108
scraping  12488  review page of product -  108
scraping  12498  review page of product -  108
scraping  12508  review page of product -  108
scraping  12518  review page of product -  108
scraping  12528  review page of product -  108
scraping  12538  review page of product -  108
scraping reviews of product -  109
scraping  12548  review page of product -  109
scraping  12558  review page of product -  109
scraping  12568  review page of product -  109
scraping  12578  review page of product -  109
scraping  12588  review page of product -  109
scraping  12598  review page of product -  109
scraping  12608  review page of product -  109
scraping  12618  review page of product -  109
scraping  12628  review page of product -  109
scraping  12638  review page of product -  109
scraping  12648  review page of product -  109
scraping  12658  review page of product -  109
scraping  12668  review page of product -  109
scraping  12678  review p

In [43]:
df.to_csv('scraped_data2.csv', index=False)